# Running queries on the OMOP data

## Install required packages

Before executing the cells below, in a terminal session install the following required R packages:

`conda create -n omop-source r-tidyverse r-data.table r-dbi  r-rpostgres r-irkernel -y`

### Connect to the SQL database

In [ ]:
library(tidyverse)
library(data.table)
library(glue)
library(DBI)
library(RPostgres)

library(tidyverse)
library(data.table)
library(glue)
library(DBI)
library(RPostgres)

DBNAME <- #<Add here relevant password>
HOST <- #<Add here relevant password>
PORT <- #<Add here relevant port>
PASSWORD <-  #<Add here relevant password>
USER <- 'jupyter_notebook'

connection <- DBI::dbConnect(
    RPostgres::Postgres(),
    dbname = $DBNAME,
    host = $HOST,
    port = $PORT,
    password = $PASSWORD,
    user = $USER,
    )

# Query 1 - Database count
This is a query that provides the total number of participants

In [ ]:
omop_query <- "
    select count(distinct o.person_id) as count
    from omop_data.person o
    limit 10
"

all_dataset_demo <- dbGetQuery(connection, glue(omop_query, schema = "omop_data"))

all_dataset_demo

In [ ]:
omop_query <- "
    select count(distinct o.person_id) as count
    from omop_data.person o
    limit 10
"

all_dataset_demo <- dbGetQuery(connection, glue(omop_query, schema = "omop_data"))

all_dataset_demo

# Query 2 - Total count stratified by gender

In [ ]:
omop_query <- "
    select count(distinct p.person_id) as count, c.concept_name, c.concept_id
    from omop_data.person p
    left join omop_data.concept c on c.concept_id = p.gender_concept_id
    group by c.concept_name, c.concept_id
    order by count desc
    limit 10
"

all_dataset_demo <- dbGetQuery(connection, glue(omop_query, schema = "omop_data"))

all_dataset_demo

# Query 3 - Number of participants in the database with Acure Renal Failure Syndrome

In [ ]:
omop_query <- "
    select count(distinct o.person_id) as count, o.condition_concept_id
    from omop_data.condition_occurrence o
    where o.condition_concept_id = '197320'
    group by o.condition_concept_id
    order by count desc
"

all_dataset_demo <- dbGetQuery(connection, glue(omop_query, schema = "omop_data"))

all_dataset_demo

# Query 4 - Comorbidities
This is a simple query that provides the comorbidities with participants diagnosed with "Acute renal failure syndrome" OMOP concept_id = 197320. (30 days window before the first diagnosis with 197320)

In [ ]:
source_query <- "
With CTE_index as (
select min(co.condition_start_date) as index_date,co.person_id
from omop_data.condition_occurrence co
left join omop_data.concept c on co.condition_concept_id = c.concept_id
where c.concept_id = 197320
group by co.person_id
),
CTE_comorbidities as (
select count(distinct co.person_id) as comorbities_num, c.concept_name, c.concept_id, co.condition_source_value
from CTE_index as fe
left join omop_data.condition_occurrence co on co.person_id = fe.person_id and co.condition_start_date between fe.index_date - 30 and fe.index_date -1
left join omop_data.concept c on c.concept_id = co.condition_concept_id
where c.concept_id != '0'
group by c.concept_name, c.concept_id, co.condition_source_value
order by comorbities_num desc
)
select * from CTE_comorbidities
limit 10
"

query <- dbGetQuery(connection, glue(source_query, schema = "source_data"))

query


# Query 5 - Adverse Events
This is a query that provides the adverse events of participants initiating 'Carboplatin' with OMOP concept_id = 1344905 (30 days window after the first exposure to 1344905)

In [ ]:
source_query <- "
With CTE_index as (
select min(co.drug_exposure_start_date) as index_date,co.person_id
from omop_data.drug_exposure co
left join omop_data.concept c on co.drug_concept_id = c.concept_id
where c.concept_id = 1344905
group by co.person_id
),
CTE_adverse_events as (
select count(distinct co.person_id) as comorbities_num, c.concept_name, c.concept_id, co.condition_source_value
from CTE_index as fe
left join omop_data.condition_occurrence co on co.person_id = fe.person_id and co.condition_start_date between fe.index_date +1 and fe.index_date + 30
left join omop_data.concept c on c.concept_id = co.condition_concept_id
where c.concept_id != '0'
group by c.concept_name, c.concept_id, co.condition_source_value
order by comorbities_num desc
)
select * from CTE_adverse_events
limit 10
"

query <- dbGetQuery(connection, glue(source_query, schema = "source_data"))

query
